In [1]:
import requests
import json
import pandas as pd 

# Informations du client
client_id = 'm7eccvja28j882ndzn4h46fuh' # A modifier
client_secret = 'nu228RpY9M' # A modifier

# Obtenir le jeton d'accès
url_token = "https://api.lufthansa.com/v1/oauth/token"
headers_token = {
    'Content-Type': 'application/x-www-form-urlencoded'
}
data_token = {
    'client_id': client_id,
    'client_secret': client_secret,
    'grant_type': 'client_credentials'
}

response_token = requests.post(url_token, headers=headers_token, data=data_token)
if response_token.status_code == 200:
    token_info = response_token.json()
    access_token = token_info['access_token']



headers_common = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json'
}


In [49]:
url = "https://api.lufthansa.com/v1/references/airports/"

all_cities_code = []
all_countries_code = []
all_airports = []
all_location_type = []
all_airports_code = []
all_longitude = []
all_latitude = []

limit = 100
offset = 0
while True:
    params = {
        "limit": limit,
        "offset": offset,
        "lang": "en"
    }

    response = requests.get(url, headers=headers_common, params=params)
    data = response.json()
    
    airports = data.get('AirportResource', {}).get('Airports', {}).get('Airport', [])

    for airport in airports:
        
        all_cities_code.append(airport.get('CityCode'))
        all_airports.append(airport.get('Names',{}).get('Name',[]).get('$'))
        all_countries_code.append(airport.get('CountryCode'))
        all_airports_code.append(airport.get('AirportCode'))
        all_location_type.append(airport.get('LocationType'))
        all_longitude.append(airport.get('Position',{}).get('Coordinate',{}).get('Longitude'))
        all_latitude.append(airport.get('Position',{}).get('Coordinate',{}).get('Latitude'))
        
    offset += limit
    
    if len(airports) != limit:
        break 

In [50]:
data = {
    'CityCode' : all_cities_code,
    'Name' : all_airports,
    'CountryCode': all_countries_code,
    'AirportCode': all_airports_code,
    'LocationType': all_location_type,
    'Longitude': all_longitude,
    'Latitude': all_latitude
}

airports = pd.DataFrame(data)
airports

,CityCode,Name,CountryCode,AirportCode,LocationType,Longitude,Latitude
0,AAL,Aalborg,DK,AAL,Airport,9.8492,57.0928
1,AAR,Aarhus,DK,AAR,Airport,10.6194,56.3039
2,ABE,Allentown/Bethl,US,ABE,Airport,-75.4408,40.6522
3,ABJ,Abidjan,CI,ABJ,Airport,-3.9264,5.2614
4,ABK,Kabri Dehar,ET,ABK,Airport,44.2553,6.7339
...,...,...,...,...,...,...,...
1462,ZTH,Zakynthos,GR,ZTH,Airport,20.8842,37.7508
1463,ZUH,Zhuhai,CN,ZUH,Airport,113.3761,22.0067
1464,STR,Stuttgart - Railway Station,DE,ZWS,RailwayStation,9.1814,48.7836
1465,ZYI,Zunyi Xinzhou,CN,ZYI,Airport,107.2461,27.8111


In [31]:
url_countries = "https://api.lufthansa.com/v1/references/countries/"

all_countries_code = []
all_countries_name = []
all_zone_code = []

limit = 100
offset = 0

while True:
    params = {
        "limit": limit,
        "offset": offset,
        "lang": "en"
    }

    response = requests.get(url_countries, headers=headers_common, params=params)
    data = response.json()

    countries = data.get('CountryResource', {}).get('Countries', {}).get('Country', [])

    for country in countries:
        all_countries_code.append(country.get('CountryCode'))
        all_countries_name.append(country.get('Names',{}).get('Name',{}).get('$'))
        all_zone_code.append(country.get('ZoneCode')) 
    
    if len(countries) != limit:
        break 
        
    offset += limit

In [32]:
data = {
    'CountryCode' : all_countries_code,
    'Name' : all_countries_name,
    'ZoneCode' : all_zone_code
}

countries = pd.DataFrame(data)
countries

,CountryCode,Name,ZoneCode
0,AD,Andorra,EUR
1,AE,United Arab Emirates,MEA
2,AF,Afghanistan,ASI
3,AG,Antigua & Barbuda,NAM
4,AI,Anguilla,NAM
...,...,...,...
236,YT,Mayotte,AFR
237,ZA,South Africa,AFR
238,ZM,Zambia,AFR
239,ZR,Zaire,AFR


In [37]:
url_cities = "https://api.lufthansa.com/v1/references/cities/"


all_cities_code = []
all_countries_code = []
all_cities = []
all_longitude = []
all_latitude = []

limit = 100
offset = 0

while True:
    params = {
        "limit": limit,
        "offset": offset,
        "lang": "en"
    }

    response = requests.get(url_cities, headers=headers_common, params=params)
    data = response.json()

    cities = data.get('CityResource', {}).get('Cities', {}).get('City', {})
    
    for city in cities:
        all_cities.append(city.get('Names',{}).get('Name',[]).get('$'))
        all_cities_code.append(city.get('CityCode'))
        all_countries_code.append(city.get('CountryCode'))
        all_longitude.append(city.get('Position',{}).get('Coordinate',{}).get('Longitude'))
        all_latitude.append(city.get('Position',{}).get('Coordinate',{}).get('Latitude'))

    offset += limit

    if len(cities) < limit:
        break

In [38]:
data = {
    'CityCode' : all_cities_code,
    'Name' : all_cities,
    'CountryCode': all_countries_code,
    'Longitude': all_longitude,
    'Latitude': all_latitude
}

cities = pd.DataFrame(data)
cities

,CityCode,Name,CountryCode,AirportCode,Longitude,Latitude
0,AAA,Anaa,PF,None,-144.491667,-16.645000
1,AAB,Arrabury,AU,None,NaN,NaN
2,AAC,Al Arish,EG,None,33.835833,31.073333
3,AAE,Annaba,DZ,None,7.809444,36.822222
4,AAF,Apalachicola,US,None,-84.972778,29.727500
...,...,...,...,...,...,...
8391,ZYI,Zunyi,CN,ZYI,NaN,NaN
8392,ZYL,Sylhet,BD,None,91.871389,24.961111
8393,ZZC,New Carrollton,US,None,NaN,NaN
8394,ZZU,Mzuzu,MW,None,34.014167,-10.552222


In [ ]:
url_airlines = "https://api.lufthansa.com/v1/references/airlines/"

all_airlines_id = []
all_airlines = []

limit = 100
offset = 0

while True:
    params = {
        "limit": limit,
        "offset": offset,
        "lang": "en"
    }

    response = requests.get(url_airlines, headers=headers_common, params=params)
    data = response.json()

    airlines = data.get('AirlineResource', {}).get('Airlines', {}).get('Airline', {})
    print(len(airlines))
    
    for airline in airlines:
        all_airlines.append(airline.get('Names',{}).get('Name',{}).get('$'))
        all_airlines_id.append(airline.get('AirlineID'))

    offset += limit
 
    if len(airlines) < limit:
        break

In [42]:
data = {
    'AirlineID' : all_airlines_id,
    'Name' : all_airlines
}

airlines = pd.DataFrame(data)
airlines

,AirlineID,Name
0,0,AirOneAtlantic
1,0A,AmberAir
2,0B,BlueAir
3,0D,DarwinAirline
4,0J,Jetclub
...,...,...
1076,ZY,AdaAir
1077,ZZ,Zz
1078,\N,Avilu
1079,МИ,KMV


In [43]:
url_aircraft = "https://api.lufthansa.com/v1/references/aircraft/"

all_aircraft = []
limit = 100
offset = 0

while True:
    params = {
        "limit": limit,
        "offset": offset,
        "lang": "en"
    }

    response = requests.get(url_aircraft, headers=headers_common, params=params)
    data = response.json()

    aircraft = data.get('AircraftResource', {}).get('AircraftSummaries', {}).get('AircraftSummary', {})
    
    for Airline in aircraft:
        all_aircraft.append(Airline.get('AircraftCode'))

    offset += limit
 
    if len(aircraft) < limit:
        break

In [44]:
data = {
    'Name' : all_aircraft,
}

aircrafts = pd.DataFrame(data)
aircrafts

,Name
0,100
1,141
2,142
3,143
4,146
...,...
486,YK2
487,YK4
488,YN2
489,YN7


In [ ]:
# Fonction pour compter les valeurs manquantes dans un DataFrame
def count_missing_values(df, df_name):
    missing_values = df.isna().sum()
    print(f"Valeurs manquantes dans {df_name}:")
    print(missing_values)
    print("\n")

# Compter les valeurs manquantes pour chaque DataFrame
count_missing_values(countries, 'countries')
count_missing_values(cities, 'cities')
count_missing_values(aircrafts, 'aircrafts')
count_missing_values(airlines, 'airlines')
count_missing_values(airports, 'airports')


In [ ]:
# Remplacer les valeurs manquantes

countries['CountryCode'].fillna('Unknown', inplace=True)  
countries['ZoneCode'].fillna('Unknown', inplace=True)     

cities['CountryCode'].fillna('Unknown', inplace=True)  
cities['Longitude'].fillna(0, inplace=True)
cities['Latitude'].fillna(0, inplace=True) 

airports['CountryCode'].fillna('Unknown', inplace=True)

airlines['AirlineID'].fillna('Unknown', inplace=True) 


In [ ]:
import psycopg2
from psycopg2.extras import execute_values

# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    dbname="airport_management",
    user="postgres",
    password="*****", # mettez votre code
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

In [ ]:
# Fonction pour insérer un DataFrame dans une table PostgreSQL
def insert_dataframe(df, table_name):
    # Convertir le DataFrame en liste de tuples
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query = f"INSERT INTO {table_name}({cols}) VALUES %s"
    execute_values(cursor, query, tuples)
    conn.commit()


In [ ]:
# Insérer les DataFrames dans les tables PostgreSQL
insert_dataframe(countries, 'Countries')
insert_dataframe(cities, 'Cities')
insert_dataframe(airports, 'Airports')
insert_dataframe(airlines, 'Airlines')
insert_dataframe(aircrafts, 'Aircrafts')

# Fermer la connexion
cursor.close()
conn.close()
